aws

In [2]:
from deepface import DeepFace
import os
import shutil
import matplotlib
import PIL
import xlsxwriter
import pandas as pd
import numpy as np
import openpyxl 
import cv2
import boto3
from botocore.exceptions import ClientError
import logging

In [5]:
import boto3

def compare_faces(sourceFile, targetFile):

    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')

    imageSource = open(sourceFile, 'rb')
    imageTarget = open(targetFile, 'rb')

    response = client.compare_faces(SimilarityThreshold=80,
    SourceImage={'Bytes': imageSource.read()},
    TargetImage={'Bytes': imageTarget.read()})

    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']
        similarity = str(faceMatch['Similarity'])
        print('The face at ' +
        str(position['Left']) + ' ' +
        str(position['Top']) +
        ' matches with ' + similarity + '% confidence')

    imageSource.close()
    imageTarget.close()
    num_of_matches = len(response['FaceMatches'])
    return num_of_matches


# source_file = ''
# target_file = ''
# face_matches = compare_faces(source_file, target_file)
# print("Face matches: " + str(face_matches))



In [6]:
import boto3

def list_faces_in_collection(collection_id):
    # Initialize the AWS Rekognition client with the 'eu-central-1' region
    rekognition = boto3.client('rekognition', region_name='eu-central-1')

    try:
        # List faces in the specified collection
        response = rekognition.list_faces(CollectionId=collection_id)

        # Print the details of each face, including ExternalImageId if available
        if 'Faces' in response:
            faces = response['Faces']
            for face in faces:
                print(f"Face ID: {face['FaceId']}")
                external_image_id = face.get('ExternalImageId', 'N/A')
                print(f"External Image ID: {external_image_id}")
                print(f"Confidence: {face['Confidence']}")
                print()
        else:
            print("No faces found in the collection.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage:
# collection_id = 'sport_players'  # Replace with your collection ID
# list_faces_in_collection(collection_id)


create a collection

In [7]:
def create_collection(collection_id):
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')

    try:
        # Check if the collection already exists
        existing_collections = client.list_collections()
        if collection_id in existing_collections['CollectionIds']:
            print(f'Collection {collection_id} already exists. Skipping creation.')
        else:
            # Create a new collection
            print('Creating collection: ' + collection_id)
            response = client.create_collection(CollectionId=collection_id)
            print('Collection ARN: ' + response['CollectionArn'])
            print('Status code: ' + str(response['StatusCode']))
            print('Done...')
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
            print(f'Collection {collection_id} already exists. Skipping creation.')
        else:
            print(f'Error creating collection: {e}')


#example of usage:

# collection_id = 'footcall_club_test'
# create_collection(collection_id)

Collection footcall_club_test already exists. Skipping creation.


add face to a collection

In [8]:
def add_faces_to_collection(bucket, photo, collection_id):

    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')

    response = client.index_faces(CollectionId=collection_id,
    Image={'S3Object': {'Bucket': bucket, 'Name':
    photo}},
    ExternalImageId=photo,
    MaxFaces=1,
    QualityFilter="AUTO",
    DetectionAttributes=['ALL'])

    print('Results for ' + photo)
    print('Faces indexed:')
    for faceRecord in response['FaceRecords']:
        print(' Face ID: ' + faceRecord['Face']['FaceId'])
        print(' Location: {}'.format(faceRecord['Face']['BoundingBox']))

    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print(' ' + reason)
    return len(response['FaceRecords'])


#example of usage

# bucket = 'verycleanbucket15'
# collection_id = 'footcall_club_test'
# photo = 'neymar_jr_4.jpg'

# indexed_faces_count = add_faces_to_collection(bucket, photo, collection_id)
# print("Faces indexed count: " + str(indexed_faces_count))

display faces in a collection

In [ ]:
import boto3

def list_faces_in_collection(collection_id):
    maxResults = 2
    faces_count = 0
    tokens = True

    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')
    response = client.list_faces(CollectionId=collection_id,
    MaxResults=maxResults)

    print('Faces in collection ' + collection_id)

    while tokens:
        faces = response['Faces']

        for face in faces:
            print(face)
            faces_count += 1
        if 'NextToken' in response:
            nextToken = response['NextToken']
            response = client.list_faces(CollectionId=collection_id,NextToken=nextToken, MaxResults=maxResults)
        else:
            tokens = False
    return faces_count


#example of usage

# collection_id = 'giroudgriezmanmbappe'
# faces_count = list_faces_in_collection(collection_id)
# print("faces count: " + str(faces_count))


creating a user

In [10]:

import boto3
from botocore.exceptions import ClientError
import logging

def create_user(collection_id, user_id):
    logger = logging.getLogger(__name__)
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')
    
    # Check if the user already exists in the collection
    existing_users = client.list_users(CollectionId=collection_id)
    
    # Extract the user IDs from the existing users
    existing_user_ids = [user['UserId'] for user in existing_users['Users']]
    
    if user_id in existing_user_ids:
        logger.info(f'User {user_id} already exists in the collection. Skipping creation.')
        return
    
    try:
        logger.info(f'Creating user: {collection_id}, {user_id}')
        client.create_user(
            CollectionId=collection_id,
            UserId=user_id)
    except ClientError:
        logger.exception(f'Failed to create user with given user id: {user_id}')
        raise

"""
Finds faces in the specified image, indexes them, and stores them in the
collection.

:param collection_id: The ID of the collection where the indexed faces will be
stored at.
:param image_file_name: The image file location that will be used by indexFaces
call.

:return: The indexFaces response
"""

# collection_id = "footcall_club_test"
# user_id = "lionel_messi"
# create_user(collection_id, user_id)



deleting a user

In [11]:
logger = logging.getLogger(__name__)
session = boto3.Session(profile_name='default')
client = session.client('rekognition')

def delete_user(collection_id, user_id):

    logger.info(f'Deleting user: {collection_id}, {user_id}')
    try:
        client.delete_user(
        CollectionId=collection_id,
        UserId=user_id
        )
    except ClientError:
        logger.exception(f'Failed to delete user with given user id:{user_id}')
        raise 




#example of usage

# collection_id = "football_club_test"
# user_id = "lionel_messi"
# delete_user(collection_id, user_id)



associate users

In [12]:
from botocore.exceptions import ClientError
import boto3
import logging

logger = logging.getLogger(__name__)
session = boto3.Session(profile_name='default')
client = session.client('rekognition')

def associate_faces(collection_id, user_id, face_ids):

    logger.info(f'Associating faces to user: {user_id}, {face_ids}')
    try:
        response = client.associate_faces(
        CollectionId=collection_id,
        UserId=user_id,
        FaceIds=face_ids
        )
        print(f'- associated {len(response["AssociatedFaces"])} faces')
    except ClientError:
        logger.exception("Failed to associate faces to the given user")
        raise 
    
    else:
        print(response)
        return response
    """
    Associate stored faces within collection to the given user

    :param collection_id: The ID of the collection where user and faces are
    stored.
    :param user_id: The ID of the user that we want to associate faces to
    :param face_ids: The list of face IDs to be associated to the given user

    :return: response of AssociateFaces API
    """


#example of usage:
# face_id1 =  '4b8a9a9d-6878-4aa4-b654-26376518f2f1'
# face_id2 = '8cb3581d-3cf5-47a1-bc04-d2ee816272bd'
# face_id3 = '5cabdac7-b358-425b-8e8c-8f55ca704cea'

# face_ids = [face_id3]
# collection_id = "footcall_club_test"
# user_id = "neymar_jr"
# associate_faces(collection_id, user_id, face_ids) 


search faces

In [14]:
import boto3
session = boto3.Session(profile_name='default')
client = session.client('rekognition')

def search_face_in_collection(face_id, collection_id):
    threshold = 90
    max_faces = 2



    response = client.search_faces(CollectionId=collection_id,
    FaceId=face_id,
    FaceMatchThreshold=threshold,
    MaxFaces=max_faces)

    face_matches = response['FaceMatches']
    print('Matching faces')
    for match in face_matches:
        print('FaceId:' + match['Face']['FaceId'])
        print('Similarity: ' + "{:.2f}".format(match['Similarity']) + "%")

    return len(face_matches)


#example od usage

# face_id = 'b1754f25-acdc-4edd-a93c-367dd94ad6a3'
# collection_id = 'collection-id'

# faces = []
# faces.append(face_id)

# faces_count = search_face_in_collection(face_id, collection_id)
# print("faces found: " + str(faces_count))


search for users

In [ ]:
import boto3
from botocore.exceptions import ClientError
import logging
logger = logging.getLogger(__name__)
session = boto3.Session(profile_name='default')
client = session.client('rekognition')
def search_users_by_face_id(collection_id, face_id):
    logger.info(f'Searching for users using a face-id: {face_id}')
    try:
        response = client.search_users(
        CollectionId=collection_id,
        FaceId=face_id
        )
        print(f'- found {len(response["UserMatches"])} matches')
        print([f'- {x["User"]["UserId"]} - {x["Similarity"]}%' for x in
        response["UserMatches"]])
    except ClientError:
        logger.exception(f'Failed to perform SearchUsers with given face id:{face_id}')
        raise
    else:
        print(response)
        return response
    """
    SearchUsers operation with face ID provided as the search source

    :param collection_id: The ID of the collection where user and faces are
    stored.
    :param face_id: The ID of the face in the collection to search for.

    :return: response of SearchUsers API
    """

    """
    SearchUsers operation with user ID provided as the search source

    :param collection_id: The ID of the collection where user and faces are
    stored.
    :param user_id: The ID of the user in the collection to search for.

    :return: response of SearchUsers API
    """

#example of usage

# collection_id = "collection-id"
# # user_id = "cristiano_ronaldo"
# face_id = '4711a37d-9fed-4777-8e17-e5b26536a8e2'  #neymar
# search_users_by_face_id(collection_id, face_id)
# # search_users_by_user_id(collection_id, user_id)



get user name from face id

In [16]:
import boto3
from botocore.exceptions import ClientError

def get_user_from_face_id(collection_id, face_id, region_name='eu-central-1'):
    try:
        # Initialize the Rekognition client with the specified region
        client = boto3.client('rekognition', region_name=region_name)

        # Search for the user associated with the face ID
        response = client.search_faces(
            CollectionId=collection_id,
            FaceId=face_id
        )

        if 'FaceMatches' in response:
            for match in response['FaceMatches']:
                user_id = match['Face']['ExternalImageId']
                similarity = match['Similarity']
                return f"User ID: {user_id}, Similarity: {similarity}%"

        # If no matches were found
        return "No user found for the provided face ID"

    except ClientError as e:
        # Handle any errors that occur during the search
        print("Error:", e)
        return None


# collection_id = 'collection-id'
# face_id = '257c4d32-7066-4220-b4c5-722b9fced48b'
# user = get_user_from_face_id(collection_id, face_id, region_name='eu-central-1')

# if user:
#     print(user)


get faces information

In [17]:
import boto3

def get_face_information(collection_id, face_id):
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')
    response = client.list_faces(CollectionId=collection_id)
    

    # Iterate through the faces to find the one with the matching FaceId
    for face in response['Faces']:
        if face['FaceId'] == face_id:
            return face

    return None  # Return None if the face with the given FaceId is not found

# # Example of usage
# collection_id = 'collection-id'
# face_id = '257c4d32-7066-4220-b4c5-722b9fced48b'

# face_info = get_face_information(collection_id, face_id)

# if face_info:
#     print('Face Information:')
#     print(face_info)
# else:
#     print(f'Face with FaceId {face_id} not found in the collection.')


get user id

In [19]:


def get_user_information(collection_id, face_id):
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')
    response = client.list_faces(CollectionId=collection_id)

    # Iterate through the faces to find the one with the matching FaceId
    for face in response['Faces']:
        if face['FaceId'] == face_id:
            if 'UserId' in face:
                return face['UserId']
            else:
                return 'Unowkn -yet'

    return None  # Return None if the face with the given FaceId is not found

#example:
# collection_id = 'collection-id'
# face_id = '257c4d32-7066-4220-b4c5-722b9fced48b'

# User = get_user_information(collection_id,face_id)
# print(User)


adds one face to the collection and returns its face id

In [20]:

def add_face_to_collection_return_faceID(bucket, folder, photo, collection_id):
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')

    object_key = f'{folder}{photo}'  # Combine the folder name and photo name
    
    try:
        response = client.index_faces(
            CollectionId=collection_id,
            Image={'S3Object': {'Bucket': bucket, 'Name': object_key}},
            ExternalImageId=photo,
            MaxFaces=1,
            QualityFilter="AUTO",
            DetectionAttributes=['ALL']
        )

        face_id = None  # Initialize to None in case no faces are indexed

        for faceRecord in response.get('FaceRecords', []):
            face_id = faceRecord.get('Face', {}).get('FaceId')

        if face_id is None:
            raise Exception(f'No face found in the image- problem with {photo}')

        return face_id
    except ClientError as e:
        raise Exception(f'Error adding face to collection: {e}- problem with {photo}')

#example of usage:

# bucket = 'giroudgriezmanmbappe'
# collection_id = 'giroudgriezmanmbappe'
# folder = 'mbappe/'
# photo = 'mbappe5.jpg'

# face_id = add_face_to_collection_return_faceID(bucket,folder, photo, collection_id)
# print(face_id)




get file names in a bucket

In [21]:
def get_all_files_in_folder(bucket_name, folder_name):
  """Prints all the files inside a folder in an S3 bucket.

  Args:
    bucket_name: The name of the S3 bucket.
    folder_name: The name of the folder inside the S3 bucket.

  Returns:
    A list of all the names of the files inside the folder in the S3 bucket.
  """

  s3 = boto3.client('s3')

  paginator = s3.get_paginator('list_objects_v2')

  response_iterator = paginator.paginate(Bucket=bucket_name, Prefix=folder_name)

  file_names = []

  for response in response_iterator:
    for obj in response['Contents'][1:]:
      
      string = obj['Key']
      parts = string.split('/')
      file_names.append(parts[-1])

  return file_names

#example of usage
# bucket_name = 'test-for-download'

# folder_name = '1/'

# get_all_files_in_folder(bucket_name, folder_name)


accosiate list of faces to user

In [22]:

def associate_faces_to_user(collection_id, user_id, face_ids):
    logger = logging.getLogger(__name__)
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition')

    logger.info(f'Associating faces to user: {user_id}, {face_ids}')
    try:
        response = client.associate_faces(
        CollectionId=collection_id,
        UserId=user_id,
        FaceIds=face_ids
        )
        # print(f'- associated {len(response["AssociatedFaces"])} faces')
    except ClientError:
        logger.exception("Failed to associate faces to the given user")
        raise 
    
    else:
        # print(response)
        return response
    """
    Associate stored faces within collection to the given user

    :param collection_id: The ID of the collection where user and faces are
    stored.
    :param user_id: The ID of the user that we want to associate faces to
    :param face_ids: The list of face IDs to be associated to the given user

    :return: response of AssociateFaces API
    """


#example of usage:
# face_id1 =  '4b8a9a9d-6878-4aa4-b654-26376518f2f1'
# # face_id2 = '8cb3581d-3cf5-47a1-bc04-d2ee816272bd'
# face_id3 = '5cabdac7-b358-425b-8e8c-8f55ca704cea'

# face_ids = [face_id3]
# collection_id = "footcall_club_test"
# user_id = "neymar_jr"
# associate_faces_to_user(collection_id, user_id, face_ids) 


create a user from folder

In [23]:
def create_user_from_folder(collection_id, bucket, folder):
    user_id = folder[:-1]
    pre_face_id_list = []
    
    # Gets the names of all the images in the folder
    images = get_all_files_in_folder(bucket_name=bucket, folder_name=folder)
    
    for image in images:
        try:
            face_id = add_face_to_collection_return_faceID(bucket=bucket, folder=folder, photo=image, collection_id=collection_id)
            pre_face_id_list.append(face_id)
        except Exception as e:
            print(f"Skipping image {image} due to error: {e}")
    
    if not pre_face_id_list:
        # Skip user creation if there are no valid faces
        return
    
    # Create a user with a name of that folder
    create_user(collection_id=collection_id, user_id=user_id)
    
    # Associate the list of faces to that user
    face_id_list = [id for id in pre_face_id_list if id is not None]
    associate_faces_to_user(collection_id=collection_id, user_id=user_id, face_ids=face_id_list)


#example of usege:

# collection_id = "football_club"
# bucket = 'test-for-download'
# folder = 'lionel_messi/'

# create_user_from_folder(collection_id, bucket, folder)


get folder's names

In [24]:
def get_all_folders_in_bucket(bucket_name):
  """Gets all the folders in an S3 bucket.

  Args:
    bucket_name: The name of the S3 bucket.

  Returns:
    A list of all the folders in the S3 bucket.
  """

  s3 = boto3.client('s3')

  paginator = s3.get_paginator('list_objects_v2')

  response_iterator = paginator.paginate(Bucket=bucket_name, Prefix='', Delimiter='/')

  folders = []

  for response in response_iterator:
    for obj in response['CommonPrefixes']:
      folders.append(obj['Prefix'])

  return folders


# get_all_folders_in_bucket('test-for-download')

creation of a dataset collection

In [28]:
def create_collection_from_bucket(bucket):
    collection_id = bucket
    create_collection(collection_id= collection_id)
    list_of_folders = get_all_folders_in_bucket(bucket)
    for folder in list_of_folders:
        create_user_from_folder(collection_id,bucket,folder)
        print('success for',folder)



bucket = 'test-2jnfjdnfjdnf'

create_collection_from_bucket(bucket= bucket)


Creating collection: test-2jnfjdnfjdnf
Collection ARN: aws:rekognition:eu-central-1:949346905981:collection/test-2jnfjdnfjdnf
Status code: 200
Done...
success for eviatar_or/
success for noa_argamani/


create a bucket

In [26]:
def upload_folder_to_s3(bucket_name, folder_path, region_name='eu-central-1'):
    # Initialize an S3 client in the specified region
    s3_client = boto3.client('s3', region_name=region_name)

    # Extract the base folder name
    folder_name = os.path.basename(folder_path)

    # Create an S3 bucket (if it doesn't exist) in the specified region
    try:
        location = {'LocationConstraint': region_name}
        s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f"Bucket {bucket_name} created in region {region_name}")
    except s3_client.exceptions.BucketAlreadyOwnedByYou:
        print(f"Bucket {bucket_name} already exists and is owned by you.")
    except Exception as e:
        print(f"An error occurred while creating the bucket: {e}")
        return

    # Iterate through folders and files in the local folder
    for root, _, files in os.walk(folder_path):
        for file in files:
            local_file_path = os.path.join(root, file)
            s3_object_key = os.path.relpath(local_file_path, folder_path).replace("\\", "/")

            try:
                # Upload the local file to the specified S3 bucket
                s3_client.upload_file(local_file_path, bucket_name, f"{folder_name}/{s3_object_key}")
                print(f"Uploaded {s3_object_key} to {bucket_name} in region {region_name}")
            except Exception as e:
                print(f"An error occurred while uploading {s3_object_key}: {e}")

    print(f'Finished uploading folder - {folder_path}')

# # Example usage:
# bucket_name = 'jdjfbdskjfbsdkflll'
# folder_path =  r''
# upload_folder_to_s3(bucket_name, folder_path)
  


create data-base

In [ ]:
def create_data_base(data_base_path, bucket_name):
    # Get the absolute path to the current working directory
    current_directory = os.path.abspath(os.path.curdir)
    
    if os.path.exists(data_base_path) and os.path.isdir(data_base_path):
        subfolders = [f for f in os.listdir(data_base_path) if os.path.isdir(os.path.join(data_base_path, f))]
        if subfolders:
            for subfolder in subfolders:
                subfolder_path = os.path.relpath(os.path.join(data_base_path, subfolder), current_directory)
                upload_folder_to_s3(bucket_name, subfolder_path, region_name = 'eu-central-1')
        else:
            print("No subfolders found in", data_base_path)
    else:
        print("The specified path is not a valid folder.")

# Usage

bucket_name = 'test-2jnfjdnfjdnf'
data_base_path = r''
create_data_base(data_base_path= data_base_path , bucket_name= bucket_name)


